# <center>VIPER PTA Summer School<br><br>Overlap Reduction Function Tutorial<br><br>Author: Nima Laal

![alt text](../Image/meme.jpg "PTMCMC")

## Waiting for PTMCMC to Converge!

# Packages to Install...

In [ ]:
#!pip install corner
#!pip install nestle
#!pip install scipy
#!pip install astropy
#!pip install sympy
#!pip install statsmodels

# Loading Packages

In [ ]:
import numpy as np
from NimaDemoUtils import *
import corner, nestle, matplotlib, warnings
from itertools import combinations
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.style.use('dark_background')
warnings.filterwarnings('ignore')
colors = ["aqua", "tomato", "gold", "lawngreen", "white", "magenta", "chocolate", "orange", 'green', 'gray']

# <center> Section 1: The Simplest Case of a PTA!

## Imagining a PTA

In [ ]:
Npulsars = 150
n_realization = 1
seeds = np.linspace(10000, 10062300, n_realization, dtype = int)
disttype = 'uniform'
start_obs = 53000 #mjd units
dur_in_year = 13
end_obs = start_obs + dur_in_year * 365.25
toas = np.array([np.arange(start_obs,end_obs, 30)] * Npulsars)
toas

In [ ]:
lam, beta, pname, xiMat = PulsarDistMaker(Npulsars = Npulsars, 
                                          seed = seeds[0], 
                                          skyplot = True, 
                                          disttype = disttype)

## Injecting a GWB

In [ ]:
res_gw = np.zeros((n_realization, Npulsars, len(toas[0])))
for ii, seed in enumerate(seeds):
    print('Making Realization {0}/{1}...'.format(ii + 1, n_realization), end = '\r')
    res_gw[ii,:,:] = GWBInj(Amp = 2e-15,
           start_obs = start_obs, 
           end_obs = end_obs, 
           Npulsars = Npulsars, 
           ang = xiMat.flatten(), 
           seed = seed, 
           toas = toas)

## Extracting the Cross Correlations

## $$\large \text{ORF} \propto \left\langle {R_a R_b} \right\rangle$$

### Finding the Cross Indices

In [ ]:
Npairs = int(Npulsars * (Npulsars - 1)/2)
ab = list(combinations(np.arange(0,Npulsars),2))
ab

### Calculating $\left\langle {R_i R_j} \right\rangle$ for Each Realization in ***Time Domain***

$$\large \begin{align}
 {{\rho }_{ab}}=&\text{Time Mean}\left\{ {{R}_{i}}\left( t \right){{R}_{j}}\left( t \right) \right\} \\ 
 {{\sigma }_{ab}}=&\text{Time STD}\left\{ {{R}_{i}}\left( t \right){{R}_{j}}\left( t \right) \right\} \\ \end{align}$$


In [ ]:
mean_corr = np.zeros((n_realization, Npairs))
sig_corr = np.zeros((n_realization, Npairs))
sig_corr_better = np.zeros((n_realization, Npairs))
xiab = np.zeros(Npairs); HD_sigma = np.zeros(Npairs)
for rr in range(n_realization):
    for kk, (a,b) in enumerate(ab):  
        calc = res_gw[rr, a,:]*res_gw[rr, b,:]
        mean_corr[rr,kk] = np.mean(calc)
        sig_corr[rr,kk] = np.std(calc, ddof=1)
        sig_corr_better[rr,kk] = np.sqrt(0.5 * (HD(xiMat[a,b])**2 + 4*HD(.0000001)))
        if rr == 0: 
            xiab[kk] = xiMat[a,b] # This line extracts the angular separation value from xiMat matrix 
##Averaging Over Different Realizations
rho = np.mean(mean_corr,axis = 0)
sig = np.mean(sig_corr,axis = 0)
sig_better = np.mean(sig_corr_better,axis = 0)
xi = xiab.T #transpose for the sake of plotting!

## Have We Estimated the Standard Deviation Correctly?

In [ ]:
figg, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (5,5), dpi = 140)
plt.figure(figsize = (12,6))
plot_acf(calc, lags = len(calc) - 1 , ax = ax)
plt.show()

In [ ]:
figg, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (5,5), dpi = 140)
plt.figure(figsize = (12,6))
plot_acf(np.random.normal(loc = 0, scale = .2, size = 1000), lags = 99, ax = ax)
plt.show()

## Analyzing the Obtained Cross Correlation Values

### Finding the Normalization Constant For the Best HD Fit

In [ ]:
first_norm = 1e-15 #Making the Sampler not to deal with very small numbers!

### Method 1: Using STD Over Time as Error Estimate

In [ ]:
ii = 0; flag = True; Mod = True
while flag:
    try:
        print('{} Time Trying...'.format(ii+1))
        fullmodel = CrossModel(rho/first_norm, sig/first_norm, xi, pmin=[-10], model='findhdnorm',
                         pmax=[10])
        fullresult = nestle.sample(fullmodel.get_loglike,
                                   fullmodel.get_prior_transform,
                                   ndim = 1,
                                   npoints=300, dlogz=0.1)
        flag = False
    except:
        ii += 1
        if ii > 4: flag = False; Mod = False; print('***Modeling Failed; Plot the unbinned correlations to see why!***')
        continue

### Method 2: Using Theoretical HD Bounds as Error Estimates

In [ ]:
ii = 0; flag = True; Mod = True
while flag:
    try:
        print('{} Time Trying...'.format(ii+1))
        fullmodel = CrossModel(rho/first_norm, sig_better, xi, pmin=[50], model='bettersigma',
                         pmax=[150])
        fullresult = nestle.sample(fullmodel.get_loglike,
                                   fullmodel.get_prior_transform,
                                   ndim = 1,
                                   npoints=300, dlogz=0.1)
        flag = False
    except:
        ii += 1
        if ii > 4: flag = False; Mod = False; print('***Modeling Failed; Plot the unbinned correlations to see why!***')
        continue

In [ ]:
if Mod:
    plt.figure(dpi = 140)
    plt.hist(fullresult.samples, bins=40, weights = fullresult.weights, histtype = 'step', color = 'aqua');
    second_norm = np.average(fullresult.samples.flatten(), weights = fullresult.weights)
    plt.axvline(second_norm, label = 'mean = {}'.format(round(second_norm, 3)), ls = '--', color = 'tomato')
    plt.legend()
    plt.xlabel('Best Fit Normalization Constant\n(Second Nromalization)')
    plt.ylabel('PDF');
else:
    print('Modeling Failed')

### Binning the Calculated Cross Correlation Values (Two Different Ways...)

In [ ]:
norm = first_norm * second_norm

In [ ]:
nbins = 13 #Used in the first way of binning. Bins' width are equally spaced. Some bins may not contain any pulsar pair if your number of pulsars is low!
npairs = 66 #Used in the second way of binning. npairs is the number of pulsars per bin
#xi_mean, xi_err, rho_avg, sig_avg = binned_corr_Maker_forced(xi, rho/norm, sig/norm, nbins = nbins)

#*********Choose Between One!
xi_mean, xi_err, rho_avg, sig_avg = binned_corr_Maker_forced(xi, rho/norm, sig/norm, npairs = 500)
xi_mean, xi_err, rho_avg, sig_avg = binned_corr_Maker_forced(xi, rho/norm, sig_better, npairs = 500)

### Finally, Plotting the Cross Correlation Values

#### Unbinned Case (useful for when you only have one realization)

In [ ]:
plt.figure(dpi = 140)
plt.errorbar(xi*180/np.pi, rho/norm, yerr=sig/norm, marker='o', ls='', label = 'Estimated Correlations',alpha = .6,
                            color='aqua', capsize=4, elinewidth=1.2)
plt.ylabel('Normalized\nCross Correlation Values')
plt.xlabel('Angular Separation');
plt.title('Number of Pulsars : {0}\n Distribution Type : {1}\n Number of Realizations : {2}'.format(Npulsars, disttype, n_realization))
plt.tight_layout()
plt.show()

#### Binned Case

In [ ]:
angs = np.linspace(0.001,np.pi,100)
rho_theo = np.zeros(len(angs)); sig_theo = np.zeros(len(angs))
for ii, ang in enumerate(angs):
    rho_theo[ii] = HD(ang)
    sig_theo[ii] = np.sqrt(0.5 * (HD(ang)**2 + 4*HD(.0000001)))

In [ ]:
plt.figure(figsize = (12,6), dpi = 140)
plt.errorbar(xi_mean*180/np.pi, rho_avg, xerr=xi_err*180/np.pi, yerr=sig_avg, marker='o', ls='', label = 'Estimated Correlations',alpha = .6,
                            color='aqua', capsize=4, elinewidth=1.2)

if Mod:
    plt.plot(angs * 180/np.pi, HD(angs), lw = 2, color = 'tomato', label = 'Theoretical HD')
    plt.plot(angs * 180/np.pi, rho_theo - sig_theo, ls = '--', lw = 2, color = 'tomato', label = 'Lower Bound Theoretical HD')
    plt.plot(angs * 180/np.pi, rho_theo + sig_theo, ls = '--', lw = 2, color = 'tomato', label = 'Upper Bound Theoretical HD')
    #plt.errorbar(xi_mean*180/np.pi, rho_avg_HD, xerr=xi_err*180/np.pi, yerr=sig_avg_HD, marker='o', ls='', label = 'Theoretical Binned HD Correlations',alpha = .6,
     #                       color='tomato', capsize=4, elinewidth=1.2)
plt.legend()
plt.ylabel('Normalized ORF')
plt.xlabel('Angular Separation');
plt.title('Number of Pulsars : {0}\n Distribution Type : {1}\n Number of Realizations : {2}'.format(Npulsars, disttype, n_realization))
plt.tight_layout()
plt.savefig('../Image/Corr_{0}_{1}_{2}.png'.format(Npulsars, disttype, n_realization), dpi = 300)

## Does the above plot look good to you (can you bin the data differently and get a better looking plot)? Is HD curve (not your calculated cross correlations) an exact line with no uncertainity? What is the uncertainity on HD curve?

### Parameterizing HD

In [ ]:
def gt( x, tau, mono ) :
    cos_ang = np.cos(x)
    k = 1/2*(1-cos_ang)
    return 1/8 * (3+cos_ang) + (1-tau)*3/4*k*np.log(k) + mono

In [ ]:
taus = [-1.5, -1, 0, .5, 1.5]
ang = np.linspace(0.001,np.pi,100)
plt.figure(dpi = 140)
for ii, tau in enumerate(taus):
    if tau == -1: label = 'tau = -1 (HD)'
    else: label = 'tau = {}'.format(tau)
    plt.plot(ang, gt(ang, tau, 0),lw = 3, label = label, color = colors[ii])
plt.ylabel('Parameterized ORF')
plt.xlabel('Angular Separation');
plt.legend()
plt.tight_layout()

## Find the Best Fit!

In [ ]:
fullmodel = CrossModel(rho/norm, sig/norm, xi, pmin=[-10, -2], model='gt',
                 pmax=[10, 2])
fullresult = nestle.sample(fullmodel.get_loglike,
                           fullmodel.get_prior_transform,
                           ndim = 2,
                           npoints=600, dlogz=0.1)

In [ ]:
figg, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (5,5), dpi = 140)
fig = corner.corner(fullresult.samples, bins=40,
                 labels = ['norm', 'tau'], color = 'aqua', truths = [first_norm, -1], truth_color = 'tomato',fig = figg,
                 show_titles = True, weights=fullresult.weights)
plt.savefig('../Image/CornerCorr_{0}_{1}_{2}.png'.format(Npulsars, disttype, n_realization), dpi = 300)

## Plot this best fit!

# ***Question 1***:  Do we have more than 1 realization in reality? How is it possible to immprove the detectability of HD curve?

# ***Question 2***: keeping the number of realizations the same, what affects the detectability of correlations the most?

# <center> Section 2: Do You Know Your Integrals?

In [ ]:
from sympy import *
import sympy

## The Coordinate System

![alt text](../Image/coordinate.png "The Coordinate System")

## Setting up the Problem Using Sympy

In [ ]:
## t is theta, p is phi, and gamma is gamma (angular separation between two pulsars)
t, p, gamma = sympy.symbols('theta phi gamma')
##The Most General Z-axis (***it is the same as the negative of k_hat from the lecture notes***)
omega = [sin(t) * cos(p), sin(t) * sin(p), cos(t)]
##The Most General X-axis
m = [cos(t)*cos(p),cos(t)*sin(p),-sin(t)] 
##The Most General Y-axis
n = [-sin(p),cos(p),0]
##Location of Pulsar a
u_a = [0,0,1]
##Location of Pulsar b
u_b = [sin(gamma),0,cos(gamma)]

## Do you remember this?

$$\large \begin{align}
  & R_{a}^{A}\left( f,\hat{k} \right)=\frac{1}{2}\frac{u_{a}^{i}u_{a}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{a}}}e_{ij}^{A}\left( {\hat{k}} \right)\times \left[ ... \right] \\ 
 & R_{b}^{A}\left( f,\hat{k} \right)=\frac{1}{2}\frac{u_{b}^{i}u_{b}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{b}}}e_{ij}^{A}\left( {\hat{k}} \right)\times \left[ ... \right] \\ 
\end{align}
$$

## Make some substitutions

$$\large \begin{align}
  & B=\frac{1}{2}\frac{u_{a}^{i}u_{a}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{a}}}e_{ij}^{A}\left( {\hat{k}} \right) \\ 
 & C=\frac{1}{2}\frac{u_{b}^{i}u_{b}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{b}}}e_{ij}^{A}\left( {\hat{k}} \right) \\ 
\end{align}
$$

## Choose your Polarization Mode!

![alt text](../Image/PlusPol.gif "PlusPol")

![alt text](../Image/CrossPol.gif "CrossPol")

![alt text](../Image/BRPol.gif "LongPol")

![alt text](../Image/LongPol.gif "LongPol")

![alt text](../Image/XPol.gif "xPol")

![alt text](../Image/yPol.gif "yPol")

In [ ]:
eA = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        #eA[ii] = m[i] * m[j] - n[i] * n[j] # Plus Mode
        #eA[ii] = m[i] * n[j] + m[j] * n[i] # Cross Mode
        eA[ii] = m[i] * m[j] + n[i] * n[j] #Breathing Mode
        #eA[ii] = m[i] * omega[j] + omega[i] * m[j] #Vector Longitudenial Mode 1
        #eA[ii] = n[i] * omega[j] + omega[i] * n[j] #Vector Longitudenial Mode 2
        #eA[ii] = omega[i] * omega[j] #Scalar Longitudenial
        ii = ii+1
        
B = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        B[ii] = (u_a[i] * u_a[j]) * sympy.Rational(1,2) * 1/(1 + np.dot(u_a,omega)) * eA[ii]
        ii = ii+1

C = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        C[ii] = (u_b[i] * u_b[j]) * sympy.Rational(1,2) * 1/(1 + np.dot(u_b,omega)) * eA[ii]
        ii = ii+1
        
D = factor(np.sum(B) * np.sum(C))
print('The Integrand is: ')
D

## Integrating the above over a unit sphere gives you your ORF! Can you simplify the integrands further?

In [ ]:
eA

In [ ]:
B

In [ ]:
C

# ***Question 3***:  Try finding the integrand for each of the given polarization modes. Do you see anything unusual? Can you explain why some modes give an ORF of zero?

# ***Question 4***:  Can you do the integral for all of the given polarizations? Which one do you think is the easiest to do?